from scipy.signal import sosfiltfilt, butter

from whittaker_eilers import WhittakerSmoother

def smooth_finite_diff(x, cutoff_freq=1.0):
    # sos = butter(10, cutoff_freq, output='sos', fs=100)
    # y = sosfiltfilt(sos, x)
    # return y

    whittaker_smoother = WhittakerSmoother(lmbda=3e4, order=4, data_length=x.shape[0])
    smoothed_data = whittaker_smoother.smooth(x)
    return smoothed_data



In [ ]:
new_method_smooth_inputs = np.zeros(np.shape(not_smooth_all_inputs))

In [ ]:
%matplotlib widget

# make sure that the pressure commands are true step inputs
command_data = not_smooth_all_inputs[:, 24:].copy()

curr_command = command_data[0, :]

for i in range(command_data.shape[0]):
    if np.allclose(command_data[i, :], curr_command):
        command_data[i, :] = curr_command
    else:
        command_data[i:i+10, :] = command_data[i+10, :]
        curr_command = command_data[i, :]

new_method_smooth_inputs[:, 24:] = command_data

In [ ]:
# plot the command data
fig, axs = plt.subplots(12, 1, sharex=True, figsize=(10, 20))
fig.suptitle('Smooth vs Not Smooth')
labels = ["p0", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11"]

for i in range(12):
    axs[i].plot(command_data[start_ind:end_ind, i], label=f"{labels[i]}_cmd")
    axs[i].plot(new_method_smooth_inputs[start_ind:end_ind, i+24], label=f"{labels[i]}_smooth")
    axs[i].legend()
    axs[i].grid()

In [ ]:
pressure_data = not_smooth_all_inputs[:, :12].copy()
command_data_2 = new_method_smooth_inputs[:, 24:].copy()

# take the pressure data and if the subsequent value changes less than a threshold, set it to the previous value
# check if the command changes and let it change for 200 samples after the change

from scipy.signal import sosfiltfilt, butter
sos = butter(4, 1., output='sos', fs=100)

pressures_to_do = 12
buffer = 325
change_idx = []
curr_command = command_data_2[0, 0]

for i in range(0, not_smooth_all_inputs.shape[0]):
    if np.abs(command_data_2[i, 0] - curr_command) > .05:
        curr_command = command_data_2[i, 0]
        change_idx.append(i)



print(change_idx)
change_idx = change_idx + [not_smooth_all_inputs.shape[0]]
for j in range(pressures_to_do):
    # fix the first section to be equal to the mean
    pressure_data[:change_idx[0], j] = np.mean(pressure_data[:change_idx[0], j])

    for i in range(0, len(change_idx)-1):
        idx = change_idx[i]
        # fix the buffer around the change point to be equal to the mean
        # if idx + buffer > change_idx[i+1]:
        #     print(f"Buffer too large for {j}. {i}, change_") 
        pressure_data[idx+buffer:change_idx[i+1]+3, j] = np.mean(pressure_data[idx+buffer:change_idx[i+1], j])

        # filter all of the data between the change points
        if i != len(change_idx)-1:
            pressure_data[idx-5:idx+2, j] = pressure_data[idx-10, j]
            pressure_data[idx:change_idx[i+1], j] = sosfiltfilt(sos, pressure_data[idx:change_idx[i+1], j])
            # pressure_data[idx-5:idx+2, j] = pressure_data[idx-10, j]

            flag = True
            my_idx = idx-3
            while flag or my_idx - idx < 10:
                if np.sign(pressure_data[my_idx, j] - pressure_data[my_idx-2, j]) != np.sign(command_data_2[my_idx, j] - command_data_2[my_idx-10, j]):
                    pressure_data[my_idx, j] = pressure_data[my_idx-3, j]
                else:
                    flag = False
                my_idx += 1


        # fix the point where the command changes to be equal to the mean
        # pressure_data[idx:idx+2, j] = pressure_data[idx-10, j]
        
        # flatten the data out after the last input
        # if i == len(change_idx)-1:
        #     pressure_data[idx+1:, j] = np.mean(pressure_data[idx+1:, j])

# start_ind = pressure_data.shape[0] - 5000
# end_ind = 5000 + start_ind

start_ind = 0
end_ind = 2000
plt.close()
fig, axs = plt.subplots(pressures_to_do, 1, sharex=True, figsize=(15, pressures_to_do*2.5))
fig.suptitle('Pressure Data')
labels = ["p0", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11"]

for i in range(pressures_to_do):
    axs[i].plot(command_data_2[start_ind:end_ind, i], label=f"{labels[i]}_cmd")
    axs[i].plot(not_smooth_all_inputs[start_ind:end_ind, i], label=f"{labels[i]}_not_smooth")
    axs[i].plot(pressure_data[start_ind:end_ind, i], label=f"{labels[i]}_smooth")
    axs[i].legend()
    axs[i].grid()

new_method_smooth_inputs[:, :12] = pressure_data

In [ ]:
position_data = not_smooth_all_inputs[:, 18:24].copy()

from scipy.signal import sosfiltfilt, butter
sos = butter(4, 1.5, output='sos', fs=100)

buffer = 285
buffminues = 100

change_idx_mine = [0] + change_idx
for j in range(6):
    for i in range(len(change_idx)):
        idx = change_idx_mine[i]
        position_data[change_idx_mine[i-1]+buffer:idx, j] = np.mean(position_data[change_idx_mine[i-1]+buffer:idx, j])


for j in range(6):
    for idx in change_idx:
        if idx != change_idx[-1]:
            position_data[idx:idx+buffer, j] = sosfiltfilt(sos, position_data[idx:idx+buffer, j])

# for j in range(6):
#     position_data[:, j] = sosfiltfilt(sos, position_data[:, j])

fig, axs = plt.subplots(6, 1, sharex=True, figsize=(10, 15))

for i in range(6):
    axs[i].plot(not_smooth_all_inputs[start_ind:end_ind, i+18], label=f"pos_not_smooth")
    axs[i].plot(position_data[start_ind:end_ind, i], label=f"pos_smooth")
    axs[i].plot(new_method_smooth_inputs[start_ind:end_ind, 0]/200, label=f"p0")
    axs[i].plot(new_method_smooth_inputs[start_ind:end_ind, 1]/200, label=f"p2")
    axs[i].legend()
    axs[i].grid()

new_method_smooth_inputs[:, 18:24] = position_data

In [ ]:
# get the derivative of the position data
# velocity_data = np.zeros(position_data.shape)
# velocity_data[1:, :] = (position_data[1:, :] - position_data[:-1, :]) / 0.01
# velocity_data[0] = velocity_data[1]

import pynumdiff
import pynumdiff.optimize

# smooth and numerically differentiate the pressure data
dt = 0.01
cutoff_frequency = 1
num_data_points = 20000

def smooth_finite_diff(x, dt, cutoff_frequency, num_data_points=20000, data_start=0):
    # estimate by (a) counting real # peaks per second in the data or (b) look at power spectra and choose cutoff. 
    log_gamma = -1.6*np.log(cutoff_frequency) -0.71*np.log(dt) - 5.1
    tvgamma = np.exp(log_gamma)   

    # params, val = pynumdiff.optimize.linear_model.savgoldiff(x[data_start:num_data_points+data_start], dt, params=None, 
    #                                                                 options={'iterate': True},
    #                                                                 tvgamma=tvgamma,
    #                                                                 dxdt_truth=None)
    # print('Optimal parameters: ', params)
    params = [2, 3, 3]
    x_hat, dxdt_hat = pynumdiff.linear_model.savgoldiff(x, dt, params, options={'iterate': True})

    return x_hat, dxdt_hat


for i in range(6):
    position_hat, velocity_hat = smooth_finite_diff(new_method_smooth_inputs[:, i+18], dt, cutoff_frequency, num_data_points=num_data_points)
    new_method_smooth_inputs[:, 12+i] = velocity_hat

fig, axs = plt.subplots(6, 1, sharex=True, figsize=(10, 15))

for i in range(6):
    axs[i].plot(not_smooth_all_inputs[start_ind:end_ind, i+12], label=f"not_smooth")
    axs[i].plot(velocity_hat[start_ind:end_ind], label=f"smooth_velocity")
    axs[i].plot(new_method_smooth_inputs[start_ind:end_ind, i+18], label=f"pos")
    axs[i].plot(position_hat[start_ind:end_ind], label=f"pos_hat")
    axs[i].legend()
    axs[i].grid()

In [ ]:
# find outputs from the new_smooth_inputs

path_to_save = "/home/daniel/Documents/data/daniel_baloo_data_collection/STEPCMDS/parsed_csvs"

new_method_smooth_outputs_final = new_method_smooth_inputs[1:, :24] - new_method_smooth_inputs[:-1, :24]
new_method_smooth_inputs_final = new_method_smooth_inputs[1:, :]

print(new_method_smooth_inputs_final.shape, new_method_smooth_outputs_final.shape)

np.save(path_to_save+"/smooth_all_inputs.npy", new_method_smooth_inputs_final)
np.save(path_to_save+"/smooth_all_outputs.npy", new_method_smooth_outputs_final)

In [ ]:
# plot all of the output data
states = 24

start_ind = 0
end_ind = 10000


fig, axs = plt.subplots(states, 1, sharex=False, figsize=(15, 35))
fig.suptitle('Smooth vs Not Smooth')
labels = ["p0", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "udot0", "vdot0", "udot1", "vdot1", "udot2", "vdot2", "u0", "v0", "u1", "v1", "u2", "v2"]

for i in range(states):
    axs[i].plot(new_method_smooth_inputs_final[start_ind:end_ind, i]/150, label=f"{labels[i]}_state")
    axs[i].plot(new_method_smooth_outputs_final[start_ind:end_ind, i], label=f"{labels[i]}_output")
    axs[i].legend()
    axs[i].grid()